In [3]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle

In [4]:
# face detection
# LOad the model
face_detection_model = './models/res10_300x300_ssd_iter_140000_fp16.caffemodel'
face_detection_proto = './models/deploy.prototxt.txt'
face_descriptor = './models/openface.nn4.small2.v1.t7'

detector_model = cv2.dnn.readNetFromCaffe(face_detection_proto,face_detection_model)
descriptor_model = cv2.dnn.readNetFromTorch(face_descriptor)

In [5]:
!dir emotion_data

 Volume in drive C has no label.
 Volume Serial Number is 0434-922A

 Directory of C:\Users\hp\OneDrive\Desktop\FACE RECOGNITION\emotion_data

23-11-2023  12:39    <DIR>          .
23-11-2023  12:44    <DIR>          ..
23-11-2023  12:38    <DIR>          angry
23-11-2023  12:38    <DIR>          disgust
23-11-2023  12:38    <DIR>          fear
23-11-2023  12:38    <DIR>          happy
23-11-2023  12:38    <DIR>          neutral
23-11-2023  12:39    <DIR>          sad
23-11-2023  12:39    <DIR>          surprise
               0 File(s)              0 bytes
               9 Dir(s)  114,729,717,760 bytes free


In [6]:
# consider sample image 
img = cv2.imread('./emotion_data/angry/0.jpg')
cv2.imshow('Sample',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
def helper(image_path):
    img = cv2.imread(image_path)
# step -1 face detection
    image = img.copy()
    h,w = img.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=False,crop = False)
    # set the input
    detector_model.setInput(img_blob)
    detections = detector_model.forward()
    
    if len(detections) > 0:
        i = np.argmax(detections[0,0,:,2])
        confidence = detections[0,0,i,2]
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy) = box.astype('int')
    
            roi = image[starty:endy,startx:endx].copy()
            faceblob = cv2.dnn.blobFromImage(roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
            descriptor_model.setInput(faceblob)
            vectors = descriptor_model.forward()

            return vectors
        return None
            # just for reference draw bounding
            # img_draw = image.copy()
            # cv2.rectangle(img_draw,(startx,starty),(endx,endy),(0,255,0))
    
    # cv2.imshow('sample',image)
    # cv2.imshow('draw_image',img_draw)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [9]:
# helper('./images/Sachin Tendulkar/2200.jpg')

In [10]:
data = dict(data=[],label=[])

In [11]:
folders = os.listdir('emotion_data')
# print(folders)
for folder in folders:
    filenames = os.listdir('emotion_data/{}'.format(folder))
    for filename in filenames:
      try:
            vector = helper('./emotion_data/{}/{}'.format(folder,filename))
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print('Feature Extraction Successful')
      except:
          pass

Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Extraction Successful
Feature Ex

In [12]:
data.keys()

dict_keys(['data', 'label'])

In [13]:
pd.Series(data['label']).value_counts()

surprise    1298
fear        1294
happy       1294
neutral     1291
sad         1282
angry        990
disgust      735
Name: count, dtype: int64

In [14]:
pickle.dump(data,open('data_face_features_emotion.pickle',mode='wb'))